In [22]:
import numpy as np
import math
import scipy
from scipy.optimize import root, minimize

import matplotlib.pyplot as plt
from sympy import *
init_printing()

from sympy.plotting import plot3d_parametric_line

from mpl_toolkits.mplot3d import Axes3D 

%matplotlib notebook

from jplephem.spk import SPK
from jdcal import gcal2jd, jd2gcal

Блок программ для работы с эфемеридами:

In [23]:
def toJD(year, mont, day): #v julian
    A = gcal2jd(year,mont,day)
    A = A[0] + A[1]
    return A

def toCAL(JDate): #v nashi daty
    A = JDate - 2400000.5
    B = jd2gcal(2400000.5,A)
    return B

def Ephemeris(jd, targ, cent, UnitR, UnitV):
    
##_____EPHEM__________________________
#kernel = SPK.open('de405.bsp')
## print(kernel)
#posMoon,  velMoon  = kernel[3,301].compute_and_differentiate(2459580.5)#todaydate 
#posEarth, velEarth = kernel[3,399].compute_and_differentiate(2459580.5)#todaydate 
#position = numpy.zeros(3)
#velocity = numpy.zeros(3)
#for i in range(3):
#    position[i]= posMoon[i]-posEarth[i]
#    velocity[i]=(velMoon[i]-velEarth[i])/86400
#print("moon position = ", position, "km")
#print("moon velosity = ", velocity, "km/sec")

#Solar System Barycenter (0) -> Mercury Barycenter (1)
#Solar System Barycenter (0) ->   Venus Barycenter (2)
#Solar System Barycenter (0) ->   Earth Barycenter (3)
#Solar System Barycenter (0) ->    Mars Barycenter (4)
#Solar System Barycenter (0) -> Jupiter Barycenter (5)
#Solar System Barycenter (0) ->  Saturn Barycenter (6)
#Solar System Barycenter (0) ->  Uranus Barycenter (7)
#Solar System Barycenter (0) -> Neptune Barycenter (8)
#Solar System Barycenter (0) ->   Pluto Barycenter (9)
#Solar System Barycenter (0) ->               Sun (10)
#       Earth Barycenter (3) ->             Moon (301)
#       Earth Barycenter (3) ->            Earth (399)

#____________________________________   
    
    RV = np.zeros(6);    
    kernel = SPK.open('de405.bsp');
    
    if  0 <= targ <= 11 and cent==0 or 300 < targ < 400 and cent==3:
        posTarg, velTarg = kernel[cent,targ].compute_and_differentiate(jd); 
        
        for i in range(3):
            RV[i]   = posTarg[i]/UnitR;
            RV[3+i] = velTarg[i]/86400/UnitV;   
            
        return RV;    
    
    if targ==301 or targ==399:
        CENTR = 3;
        posTarg, velTarg = kernel[CENTR,targ].compute_and_differentiate(jd); 
    elif 0 <= targ <= 11:
        CENTR = 0;
        posTarg, velTarg = kernel[CENTR,targ].compute_and_differentiate(jd); 

    if cent==301 or cent==399:
        CENTR = 3;
        posCent, velCent = kernel[CENTR,cent].compute_and_differentiate(jd); 
    elif 0 <= cent <= 11:
        CENTR = 0;
        posCent, velCent = kernel[CENTR,cent].compute_and_differentiate(jd); 

    if  0 <= targ <= 11 and 300 < cent < 400 or 300 < targ < 400 and 0 <= cent <= 11:
        posCENTR, velCENTR = kernel[0,3].compute_and_differentiate(jd); 
        if 0 <= targ <= 11:
            for i in range(3):
                RV[i]   = (posTarg[i]-posCent[i]-posCENTR[i])/UnitR;
                RV[3+i] = (velTarg[i]-velCent[i]-velCENTR[i])/86400/UnitV; 
        elif 0 <= cent <= 11:
            for i in range(3):
                RV[i]   = (posTarg[i]+posCENTR[i]-posCent[i])/UnitR;
                RV[3+i] = (velTarg[i]+velCENTR[i]-velCent[i])/86400/UnitV;  
                
    else:
        for i in range(3):
            RV[i]   = (posTarg[i]-posCent[i])/UnitR;
            RV[3+i] = (velTarg[i]-velCent[i])/86400/UnitV;         
            
    return RV

Блок программ для решения задачи Ламберта:

In [24]:
def Cn(n,x):
    
    absx = np.abs(x)
    sqrtx = sqrt(absx)
#    print(x, absx)    
    if absx < 1.e-8:
        nf=math.factorial(n)
        cn=( 1.0+x/((n+1.0)*(n+2.0))*(-1.0 + x/((n+3.0)*(n+4.0))) )/nf
    else:
        if x > 0:
            if   n==0: cn = cos(sqrtx)
            elif n==1: cn = sin(sqrtx)/sqrtx
            elif n==2: cn = (1-cos(sqrtx))/x
            elif n==3: cn = (sqrtx-sin(sqrtx))/(x*sqrtx)         
        else:
            if   n==0: cn = cosh(sqrtx)
            elif n==1: cn = sinh(sqrtx)/sqrtx
            elif n==2: cn = (cosh(sqrtx)-1)/absx
            elif n==3: cn = (sinh(sqrtx)-sqrtx)/(absx*sqrtx) 
            
    return cn 

def U(x,ro):
    cn1x = Cn(1,x)
    cn2x = Cn(2,x)
    u = sqrt(1.0-ro*cn1x/sqrt(cn2x))
    return u

def F(x,ro):
    uxro=U(x,ro)
    cn2x = Cn(2,x)
    cn3x = Cn(3,x)    
    F = (uxro**2*cn3x/sqrt(cn2x)**3 + ro)*uxro
    return F
    
def Fsol(x,ro,SIG):
#    print(F(x,ro))
    return F(x,ro)-SIG

#def FsolX(x,ro):
    
    
def Lambert (R0,R1,dt):
    
    r0=sqrt(R0[0]**2+R0[1]**2+R0[2]**2)
    r1=sqrt(R1[0]**2+R1[1]**2+R1[2]**2)
    
    R0R1 = np.zeros(3)
    
    r0r1 = R0[0]*R1[0]+R0[1]*R1[1]+R0[2]*R1[2]  # скалярное произведение
    R0R1[0] = R0[1]*R1[2] - R0[2]*R1[1]         # векторное произведение
    R0R1[1] = R0[2]*R1[0] - R0[0]*R1[2]
    R0R1[2] = R0[0]*R1[1] - R0[1]*R1[0]
            
    fi=acos(r0r1/(r0*r1))
    
    if R0R1[2]<0:
        fi=2*np.pi-fi
        
    ro=sqrt(2.0*r0*r1)/(r0+r1)*cos(fi/2)
    SIG=dt/sqrt(r0+r1)**3
    SIGpar=1.0/3.0*(sqrt(2.0)+ro)*sqrt(1.0-sqrt(2.0)*ro)
     
    if SIG<SIGpar:        
        x0=0.0     
    else: 
        absro=abs(ro)
        um=sqrt(1.0-sqrt(2.0)*absro)
        eps0=(np.pi/(2.0/3.0*um**3 + SIG - ro*um))**(1.0/3.0)*um
        x0=4.0*(np.pi-eps0)**2
    
    xsol=scipy.optimize.root(Fsol, x0, args=(ro,SIG),method='krylov',tol=1.e-5)
    
    cn1x = Cn(1,xsol.x)
    cn2x = Cn(2,xsol.x)
    cn3x = Cn(3,xsol.x)
    uxro = U(xsol.x,ro)
        
    S=sqrt((r0+r1)/cn2x)*uxro;
        
    f=1-S**2*cn2x/r0;
    g=dt-S**3*cn3x;
    
    df=-S*cn1x/(r0*r1);
    dg=1-S**2*cn2x/r1;
    
    V0 = np.zeros(3)
    V1 = np.zeros(3)
    
    for i in range (3):
        V0[i]=1./g*(R1[i]-f*R0[i])
        V1[i]=df*R0[i]+dg*V0[i]      
    
    return V0,V1,fi

In [25]:
def Transfer(n0, nk, t0, dt, UnitR=1.0, UnitV=1.0, UnitT=1.0):
    
    Vinf0 = np.zeros(3)
    Vinfk = np.zeros(3)
    vinf = np.zeros(3)
    
    RV0 = Ephemeris(t0, 10, n0, 1, 1)
    RVk = Ephemeris(t0+dt, 10, nk, 1, 1)
       
    V0, Vk, fi = Lambert (RV0[0:3]/UnitR,RVk[0:3]/UnitR,dt/UnitT); # UnitV

    V0 = V0*UnitV
    Vk = Vk*UnitV
    
    Vinf0[0:3] = V0[0:3] - RV0[3:6]
    Vinfk[0:3] = Vk[0:3] - RVk[3:6]
    
    vinf[0] = sqrt(Vinf0[0]*Vinf0[0]+Vinf0[1]*Vinf0[1]+Vinf0[2]*Vinf0[2])
    vinf[1] = sqrt(Vinfk[0]*Vinfk[0]+Vinfk[1]*Vinfk[1]+Vinfk[2]*Vinfk[2])
    
    vinf[2] = vinf[0]+vinf[1]
    
    return vinf, Vinf0, Vinfk, V0, Vk, fi

In [26]:
def Windows(n0, nk, DT0, DT, N, M, UnitR=1.0, UnitV=1.0, UnitT=1.0):
    
    Vinf0 = np.zeros((N,M,3))
    Vinfk = np.zeros((N,M,3))
    vinf = np.zeros((N,M,3))
    V0 = np.zeros((N,M,3))
    Vk = np.zeros((N,M,3))
    fi = np.zeros((N,M))  
    
    for i in range(N):
        for j in range(M):
            vinf[i,j,0:3], Vinf0[i,j,0:3], Vinfk[i,j,0:3], V0[i,j,0:3], Vk[i,j,0:3], fi[i,j] = Transfer(n0, nk, DT0[i], DT[j], UnitR, UnitV, UnitT)
    
    return vinf, Vinf0, Vinfk, V0, Vk, fi

In [27]:
def Fun(x, n0, n1, UnitR, UnitV, UnitT):
    vinf, Vinf0, Vinfk, V0, Vk, fi = Transfer(n0, n1, x[0], x[1], UnitR, UnitV, UnitT)
    return vinf[2]

In [33]:
# константы

AU     = 1.49597870691e8;  # km
fM_Sun = 1.32712440018e11; # km^3/sec^2

UnitR = AU;                  # km
UnitV = sqrt(fM_Sun/UnitR);  # km/sec
UnitT = (UnitR/UnitV)/86400; # day  

# исходные данные

dT = 250 # day

# Дата отлёта:

jd0 = toJD(2019, 11, 29)

#Положение планет:

RV0 = Ephemeris(jd0, 10, 3, 1, 1)
print('\n Координаты планеты отправления:\n')
print(RV0[0:3], 'km')
print(RV0[3:6], 'km/s')

RVk = Ephemeris(jd0+dT, 10, 4, 1, 1)
print('\n Координаты планеты отправления:\n')
print(RVk[0:3], 'km')
print(RVk[3:6], 'km/s')

# результаты расчёта

V0, Vk, fi = Lambert (RV0[0:3]/UnitR,RVk[0:3]/UnitR,dT/UnitT); # UnitV
print('\n Отлётная и подлётная скорость:\n')
print(V0[0:3]*UnitV, 'km/s')
print(Vk[0:3]*UnitV, 'km/s')
print('\n Гиперболический избыток скорости:\n')
print(V0[0:3]*UnitV-RV0[3:9], 'km/s')
print(Vk[0:3]*UnitV-RVk[3:9], 'km/s')

vinf, Vinf0, Vinfk, V0, Vk, fi = Transfer(3, 4, jd0, dT, UnitR, UnitV, UnitT)

res = minimize(Fun, [jd0, dT], args=(3, 4, UnitR, UnitV, UnitT), method='COBYLA', tol=1.e-6)

print('\n Лучший вариант лететь ', toCAL(res.x[0]), ('с продолжительностью %6.3f дней' %res.x[1])) 

N,M = 100, 100
JD0 = np.linspace(jd0,jd0+1000,N)
DT = np.linspace(150,300,M)
#VINF, VINF0, VINFk, VV0, VVk, FI = Windows(3, 4, JD0, DT, N, M, UnitR, UnitV, UnitT)


 Координаты планеты отправления:

[ -5.95964567e+07  -1.23871650e+08  -5.36980221e+07] km
[ 27.73804531 -10.933762    -4.73984288] km/s

 Координаты планеты отправления:

[ -1.90333030e+08   7.10956316e+07   3.77455507e+07] km
[-10.33850353 -22.28158478  -9.94104223] km/s

 Отлётная и подлётная скорость:

[30.7367397023052 -2.38844755886755 -2.01889175948767] km/s
[-9.19394573128306 -17.3175633195277 -7.48052178610884] km/s

 Гиперболический избыток скорости:

[2.99869439553171 8.54531443860955 2.72095112026209] km/s
[1.14455780083865 4.96402146225049 2.46052044053739] km/s

 Лучший вариант лететь  (2020, 7, 26, 0.9102683062665164) с продолжительностью 349.784 дней


Д.З.:

1. Построить изолинии vinf от jd0 и dt
2. Определить минимум на них
3. Добавить околопланетные участки, посчитать импульсы скорости и затраты топлива

In [34]:
res

     fun: 7.4977275489424464
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 519
  status: 1
 success: True
       x: array([  2.45905741e+06,   3.49784448e+02])

In [35]:
vinf[2]